# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df


,Unnamed: 0,city,lat,lon,max temp,humidity,cloudiness,wind speed,country,date
0,0,Pascagoula,30.3658,-88.5561,63.05,27,0,6.91,US,1644359689
1,1,Punta Arenas,-53.1500,-70.9167,57.31,41,75,13.80,CL,1644359689
2,2,Ahipara,-35.1667,173.1667,79.93,72,69,10.65,NZ,1644359690
3,3,Pangnirtung,66.1451,-65.7125,-7.60,77,100,9.22,CA,1644359651
4,4,Haines Junction,60.7522,-137.5108,39.15,60,79,9.22,CA,1644359690
...,...,...,...,...,...,...,...,...,...,...
558,558,Trelew,-43.2490,-65.3051,65.21,45,67,21.77,AR,1644359882
559,559,Dunmanway,51.7167,-9.1167,50.99,95,100,8.39,IE,1644359882
560,560,Businga,3.3333,20.8833,71.98,32,79,1.07,CD,1644359883
561,561,Ushtobe,45.2514,77.9838,16.21,91,100,6.33,KZ,1644359883


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = cities_df[["lat", "lon"]].astype(float)
humidity = cities_df["humidity"].astype(float)

In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
# creating new DF for ideal weather conditions
ideal_weather_df = cities_df.loc[(cities_df['max temp'] >= 75) &
                        (cities_df['max temp'] <= 95) &
                        (cities_df['wind speed'] < 10) &
                        (cities_df['cloudiness'] < 50) &
                        (cities_df['humidity'] < 60)]
ideal_weather_df

,Unnamed: 0,city,lat,lon,max temp,humidity,cloudiness,wind speed,country,date
40,40,Makaha,21.4694,-158.2175,80.51,57,40,6.91,US,1644359701
73,73,General Roca,-39.0333,-67.5833,93.60,11,37,4.23,AR,1644359712
81,81,El Porvenir,13.7617,-87.3458,90.50,21,46,2.62,HN,1644359714
185,185,Nānākuli,21.3906,-158.1547,80.55,57,40,6.91,US,1644359747
230,230,Fuerte Olimpo,-21.0415,-57.8738,78.93,22,0,4.70,PY,1644359763
248,248,Laguna,38.4210,-121.4238,75.29,44,0,3.44,US,1644359769
275,275,Gaoua,10.2992,-3.2508,78.67,16,8,4.03,BF,1644359778
278,278,Cabo San Lucas,22.8909,-109.9124,78.62,55,0,8.05,MX,1644359655
303,303,Goya,-29.1400,-59.2626,78.80,28,0,5.21,AR,1644359786
343,343,Boma,7.0805,-2.1697,75.09,56,20,3.09,GH,1644359800


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# from 6-APIs, class 3, 05-Stu_Google_Complex
hotel_df = pd.DataFrame(ideal_weather_df, columns=['city', 'country', 'lat', 'lon'])

hotel_df['hotel name'] = ""
hotel_df

,city,country,lat,lon,hotel name
40,Makaha,US,21.4694,-158.2175,
73,General Roca,AR,-39.0333,-67.5833,
81,El Porvenir,HN,13.7617,-87.3458,
185,Nānākuli,US,21.3906,-158.1547,
230,Fuerte Olimpo,PY,-21.0415,-57.8738,
248,Laguna,US,38.4210,-121.4238,
275,Gaoua,BF,10.2992,-3.2508,
278,Cabo San Lucas,MX,22.8909,-109.9124,
303,Goya,AR,-29.1400,-59.2626,
343,Boma,GH,7.0805,-2.1697,


In [15]:
#copied directly from 6-APIs, class 3, 05-Stu_Google_Complex

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lon = row["lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    print(f"retrieving results for {index}: {row['city']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    
    results = response['results']

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        print(f"the closest hotel is {results [0] ['name']}.")
        hotel_df.loc[index, "hotel name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("--------------")

retrieving results for 40: Makaha.
the closest hotel is Hawaiian Princess Resort.
--------------
retrieving results for 73: General Roca.
the closest hotel is Hotel y Casino del Río General Roca.
--------------
retrieving results for 81: El Porvenir.
the closest hotel is Campo de Futbol Las Delicias.
--------------
retrieving results for 185: Nānākuli.
the closest hotel is Camp Pālehua.
--------------
retrieving results for 230: Fuerte Olimpo.
the closest hotel is Hospedaje DAMI.
--------------
retrieving results for 248: Laguna.
the closest hotel is Holiday Inn Express & Suites Elk Grove Central - Hwy 99, an IHG Hotel.
--------------
retrieving results for 275: Gaoua.
Missing field/result... skipping.
--------------
retrieving results for 278: Cabo San Lucas.
the closest hotel is Hotel Tesoro Los Cabos.
--------------
retrieving results for 303: Goya.
the closest hotel is Apart Costanera Goya.
--------------
retrieving results for 343: Boma.
the closest hotel is Obaatanpa Hotel.
-----

In [16]:
hotel_df

,city,country,lat,lon,hotel name
40,Makaha,US,21.4694,-158.2175,Hawaiian Princess Resort
73,General Roca,AR,-39.0333,-67.5833,Hotel y Casino del Río General Roca
81,El Porvenir,HN,13.7617,-87.3458,Campo de Futbol Las Delicias
185,Nānākuli,US,21.3906,-158.1547,Camp Pālehua
230,Fuerte Olimpo,PY,-21.0415,-57.8738,Hospedaje DAMI
248,Laguna,US,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
275,Gaoua,BF,10.2992,-3.2508,
278,Cabo San Lucas,MX,22.8909,-109.9124,Hotel Tesoro Los Cabos
303,Goya,AR,-29.1400,-59.2626,Apart Costanera Goya
343,Boma,GH,7.0805,-2.1697,Obaatanpa Hotel


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [38]:
# This code works to add the markers, but they do not show the template information when they're clicked on
hotel_layer = gmaps.symbol_layer(
            locations, fill_color='red',
            stroke_color='red', scale=2,
            info_box_content = (hotel_info))

fig = gmaps.figure(center=(13.0, -87.0), zoom_level=2)
fig.add_layer(hotel_layer)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))